In [136]:
# Imports
import numpy as np

In [137]:
# Import the tools
from sys import path
path.append('/Users/reid/dev/PythonCode/tm/tools')
# path.append('D:/PythonCode/tm/tools')
from tools import gen_inputs_outputs, logsig, lin, deriv_logsig, deriv_lin

In [138]:
# Define the size of the network
# Order by layers (input, layer1, layer2, ...)
# !!! No output layer yet, wait to gen inputs
layer_sizes = [3, 7]

In [139]:
# Gen inputs, outputs and size of the last layer
inputs, outputs, S = gen_inputs_outputs(layer_sizes[0])

In [140]:
# Append the output layer size
layer_sizes = np.array(layer_sizes + [S], dtype=int)

In [141]:
# Network shape
layer_sizes

array([3, 7, 3])

In [142]:
#####
# layer_sizes = np.array([1,2,1])

In [143]:
# Inputs
inputs

array([[0, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 1],
       [1, 0, 0],
       [1, 0, 1],
       [1, 1, 0],
       [1, 1, 1]])

In [144]:
# Outputs
outputs

array([[0, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 1],
       [1, 0, 0]])

In [145]:
# Intialize the weights and biases randomly
# Also create the list to store the outputs of each layer
# Store the outptut for each layer
# Finally create the list of the sensitivities
weights_list = []
biases_list = []
n_list = []
a_list = []
s_list = []

In [146]:
for i in range(1, len(layer_sizes)):
    weights_list.append(np.random.rand(layer_sizes[i], layer_sizes[i-1]))
    biases_list.append(np.random.rand(layer_sizes[i], 1))
    n_list.append(np.empty((layer_sizes[i], 1), dtype=float))
    a_list.append(np.empty((layer_sizes[i], 1), dtype=float))
    s_list.append(np.empty((layer_sizes[i], 1), dtype=float))

In [147]:
#####
'''weights_list = [
    np.array([[-0.27],[-0.41]]),
    np.array([[0.09, -0.17]])
]
biases_list = [
    np.array([[-0.48], [-0.13]]),
    np.array([[0.48]])
]
n_list = [
    np.array([[0], [0]]),
    np.array([[0]])
]
a_list = [
    np.array([[0], [0]]),
    np.array([[0]])
]'''

'weights_list = [\n    np.array([[-0.27],[-0.41]]),\n    np.array([[0.09, -0.17]])\n]\nbiases_list = [\n    np.array([[-0.48], [-0.13]]),\n    np.array([[0.48]])\n]\nn_list = [\n    np.array([[0], [0]]),\n    np.array([[0]])\n]\na_list = [\n    np.array([[0], [0]]),\n    np.array([[0]])\n]'

In [148]:
# Define the transfer functions
transfer_functions = [logsig, lin]

In [149]:
# Define the vectors with the derivatives of the transfer functions
# These need to be converted to dialation matrices
# but this is done after the partial derivatives are calculated
# so that the numerical values can be multiplied by the identity matrix
# makes my life easier
deriv_transfer_functions = [deriv_logsig, deriv_lin]

In [150]:
# Define a function to calculate the output of the network
# while also saving the data
# Assuming that the input is already a Rx1 matrix
def run_network(X):
    # Get the input for the first layer so that
    # no need to use X and can calculate
    # recursively
    n_list[0] = np.matmul(weights_list[0], X) + biases_list[0]
    a_list[0] = transfer_functions[0](n_list[0])

    # Calculate the rest of the outptut
    for i in range(1, len(weights_list)):
        n_list[i] = np.matmul(weights_list[i], a_list[i-1]) + biases_list[i]
        a_list[i] = transfer_functions[i](n_list[i])

In [151]:
print(run_network(inputs[1].reshape(layer_sizes[0], 1)))
####
# run_network(np.array([2], dtype=float).reshape(1, 1))

None


In [152]:
print(n_list, a_list, sep='\n\n')

[array([[1.41630058],
       [1.83069418],
       [0.82391872],
       [1.80889973],
       [1.12372829],
       [0.96094165],
       [1.19406118]]), array([[3.71177369],
       [2.63951816],
       [3.0038734 ]])]

[array([[0.80475781],
       [0.8618444 ],
       [0.69506754],
       [0.85922884],
       [0.75467962],
       [0.7233103 ],
       [0.76746662]]), array([[3.71177369],
       [2.63951816],
       [3.0038734 ]])]


In [153]:
# Calculate the senstivites for the last layer
# Write a function to fill the diagonal of the matrix
def fill_F_dot(layer_num):
    a = np.identity(len(n_list[layer_num]), float)
    np.fill_diagonal(a, deriv_transfer_functions[layer_num](n_list[layer_num]).flatten())
    return a

# Iterate through the inputs and outputs
for i in range(len(inputs)):
    # Run the network
    run_network(inputs[i].reshape(layer_sizes[0], 1))

    # Get the dialation matrix for the last layer
    F_dot = fill_F_dot(-1)
    
    # Calculate the sensitivites for the last layer
    s_list[-1] = -2 * np.matmul(F_dot, outputs[i].reshape(layer_sizes[-1], 1)-a_list[-1])

    # Update the weights and biases for the last layer
    weights_list[-1] += -np.matmul(s_list[-1], a_list[-2].T)
    biases_list[-1] += -s_list[-1]

    # Iterate through the remaining layers backwards
    for i in range(len(n_list)-2, 0, -1):
        s_list[i] = np.matmul(np.matmul(fill_F_dot(i), weights_list[i+1].T), s_list[i+1])
        weights_list[i] += -np.matmul(s_list[i], a_list[i-1].T)
        biases_list[i] += -s_list[i]

In [154]:
print(weights_list)

[array([[0.15162284, 0.74707846, 0.5501527 ],
       [0.20445297, 0.41276994, 0.86671326],
       [0.66912952, 0.60763746, 0.67631449],
       [0.20233128, 0.2029852 , 0.82205314],
       [0.39316158, 0.85341937, 0.27503746],
       [0.93600856, 0.01169406, 0.69151063],
       [0.91787049, 0.66244745, 0.71483689]]), array([[52761371.46946888, 53566136.43745805, 51817956.87049748,
        52535576.08486015, 52833675.93884794, 50767697.68598682,
        54562588.15052111],
       [35444227.19220925, 35984854.77504028, 34810456.47291906,
        35292540.61763653, 35492799.313944  , 34104910.04685382,
        36654254.95901261],
       [44435805.75074524, 45113582.39623377, 43641258.43682148,
        44245639.73907731, 44496700.15332697, 42756727.04617014,
        45952796.28607454]])]


In [155]:
print(run_network(inputs[1].reshape(layer_sizes[0], 1)))

None


In [157]:
print(a_list[-1])

[[3.45922670e+08]
 [2.32385195e+08]
 [2.91337244e+08]]
